<a href="https://colab.research.google.com/github/YannisCHUPIN/ProjetIntroTALN/blob/main/TAL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TDF

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
!pip install nltk==3.8.1
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
nltk.download('stopwords')
nltk.download('punkt')
import plotly.express as px

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train_data = pd.read_csv('./sample_data/train.csv')
test_data = pd.read_csv('./sample_data/test.csv')
stemmer = SnowballStemmer('french')
stopwords = set(stopwords.words('french'))
print(train_data.head())
print(train_data.shape,test_data.shape)

               doc_id                                              titre  \
0  recette_221358.xml  Feuilleté de saumon et de poireau, sauce aux c...   
1   recette_48656.xml                       Cake poulet/moutarde/amandes   
2   recette_30049.xml           Bûche à la truite fumée (7ème rencontre)   
3   recette_71424.xml    Gâteau au yaourt au coco sans huile de laetitia   
4  recette_217204.xml                             Crêpes au canard laqué   

             type             difficulte        cout  \
0  Plat principal                 Facile       Moyen   
1          Entrée            Très facile  Bon marché   
2          Entrée  Moyennement difficile  Assez Cher   
3         Dessert            Très facile  Bon marché   
4          Entrée  Moyennement difficile       Moyen   

                                         ingredients  \
0  - 1 gros pavé de saumon - 100 g de crevettes d...   
1  - 3 œufs - 150 g de farine - 1 sachet de levur...   
2  - 800 g de filet de truite saumonné

In [ ]:
train_data.dropna(axis=0,inplace=True)
test_data.dropna(axis=0,inplace=True)
for column in ["ingredients","recette","titre"]:
  train_data[column] = train_data[column].apply(simple_preprocess)
  test_data[column] = test_data[column].apply(simple_preprocess)
#datax_train = pd.DataFrame({'tweet':data_train[1],'stem':data_train[3]})
#datax_test = pd.DataFrame({'tweet':data_test[1],'stem':data_test[3]})
data = pd.concat([train_data, test_data])
train_data.head()

,doc_id,titre,type,difficulte,cout,ingredients,recette
8260,recette_170277.xml,[],Plat principal,Facile,Moyen,"[langude, kg, cldevinaigrd, alcool, oignonpiqu...","[eaupourladégorg, mettrlalangu, couvrd, eau, f..."
9833,recette_21519.xml,[],Dessert,Facile,Moyen,"[pâtbris, oeuf, gdesucr, gdepoudrd, amand]","[faites, lcuiràblanc, minà, thermostat, amand,..."
1454,recette_72746.xml,[meringubiensech],Dessert,Très facile,Bon marché,"[blancd, oeuf, gdesucrenpoudr, gdesucrglac, pi...","[thermostat, lenneig, mélangletoutàl, aidd, dé..."
4954,recette_38858.xml,"[sandwichjambon, menthetfet]",Plat principal,Très facile,Moyen,"[commvousvoul, engros, pasdelabaguet, encormie...","[mêlcremfraîch, menthcisel, seletpoivr, montle..."
6369,recette_167836.xml,[],Entrée,Très facile,Bon marché,"[gdemâch, oignondoux, gdelardonfum, gdelentill...","[roquet, oignonéminc, estprêt, vinblancsec]"


In [ ]:
word2vec_model = Word2Vec(train_data, vector_size=100, window=5, min_count=5, workers=4)

# Entraîner le modèle
word2vec_model.train(train_data, total_epoch=10, work=4)

In [ ]:
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)
print(train_data.shape,val_data.shape,test_data.shape)

(9978, 7) (2495, 7) (1388, 7)


In [ ]:
def preprocess_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [stemmer.stem(token.lower()) if token.lower() not in stopwords else token.lower() for token in tokens]
  return "".join(tokens)
train_data

,doc_id,titre,type,difficulte,cout,ingredients,recette
8260,recette_170277.xml,Langue de boeuf sauce piquante aux cornichons,Plat principal,Facile,Moyen,"- 1 langue de 1,5 kg - 20 cl de vinaigre d'alc...",Faire tremper la langue toute une nuit dans un...
9833,recette_21519.xml,Tarte aux raisins et amandes,Dessert,Facile,Moyen,- 1 pâte brisée - 150 g de raisins bien mûrs -...,Garnissez une tourtière avec la pâte étalée. F...
1454,recette_72746.xml,Meringues bien sèches,Dessert,Très facile,Bon marché,- 2 blancs d'oeufs - 60 g de sucre en poudre -...,"Préchauffez le four à 150°, thermostat 5. Ajou..."
4954,recette_38858.xml,"Sandwich jambon, menthe et feta",Plat principal,Très facile,Moyen,- 3 tranches de pain complet ou de seigle ou d...,Faire la petit sauce à la menthe : mêler crème...
6369,recette_167836.xml,Salade de mâche et lardons,Entrée,Très facile,Bon marché,- 300 g de mâche - quelques feuilles de roquet...,Faire revenir les lardons et les déposer chaud...
...,...,...,...,...,...,...,...
11964,recette_66759.xml,"Cake à l'orange, poisson et carottes",Entrée,Très facile,Bon marché,- 2 filets de poisson cuit - 1 orange - 2 bell...,Préchauffer le four à 180°C (thermostat 6).Cou...
5191,recette_52749.xml,Bananes au citron,Dessert,Très facile,Bon marché,- 4 bananes - 1 sachet de sucre vanillé - 1/2 ...,Couper les bananes en rondelles et les mettre ...
5390,recette_32633.xml,Moussaka au thon,Plat principal,Facile,Bon marché,- 600 g de thon au naturel (boîte) - 3 auberg...,Laver et trancher fin les aubergines dans la l...
860,recette_64624.xml,Lasagnes à la ratatouille et au fromage,Plat principal,Très facile,Bon marché,- 500 g de lasagnes - 6 tranches de jambon cui...,Emincer le demi-oignon. Peler les tomates et l...


## TF-IDF

In [ ]:
vectoriseur = TfidfVectorizer()
vectoriseur.fit(data)
lexique = vectoriseur.get_feature_names_out()
lexique # a utiliser correctement

array(['cout', 'difficulte', 'doc_id', 'ingredients', 'recette', 'titre',
       'type'], dtype=object)

## Modèle

In [ ]:
# Entraîner le modèle SVC
svc_model = SVC(kernel='linear')
svc_model.fit(X_train, y_train)

# Entraîner le modèle Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Entraîner le modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


In [ ]:
# Faire des prédictions
y_pred_svc = svc_model.predict(X_test)
y_pred_nb = nb_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)

### Métriques

In [ ]:
# Calculer les métriques
print("SVC:")
print(classification_report(y_test, y_pred_svc))

print("Naive Bayes:")
print(classification_report(y_test, y_pred_nb))

print("Random Forest:")
print(classification_report(y_test, y_pred_rf))

### Les matrices de confusion :

In [ ]:
# Fonction pour générer et visualiser la matrice de confusion
def plot_confusion_matrix(y_test, y_pred, title):
    cm = confusion_matrix(y_test, y_pred)
    fig = px.imshow(cm, text_auto=True, labels=dict(y="True Label", x="Predicted Label"),
                    x=encoder.classes_, y=encoder.classes_, title=title)
    fig.show()

# Générer et visualiser les matrices de confusion
plot_confusion_matrix(y_test, y_pred_svc, "SVC Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_nb, "Naive Bayes Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest Confusion Matrix")